**Build a Simple LLM Application with LCEL**

Here, we will see how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM applications-it is a single LLM call with prompting. \
With this one should have a high level overview of: 
* Using Large language models
* Using PromptTemplates and OutputParsers
* Using LangChain Expression Language (LCEL) to chain components together
* Debugging and tracing your application using LangSmith
* Deploying your application with LangServe

OpenAI API Key and Open-Source Models->Llama3, Gemma2, mistral->Groq API

**Groq & the Language Processing Unit** \
Groq builds fast AI inference. The Groq LPU™, AI Inference Technology, delivers exceptional compute speed, affordability, and energy efficiency at scale. 

Groq solutions are based on the Language Processing Unit (LPU), a new
category of processor. Groq is the creator of the LPU and built it from the ground up to meet the unique characteristics and needs of AI. 

LPUs run Large Language Models (LLMs) at substantially faster speeds and, on an architectural level, up to 10x better energy efficiency compared to GPUs.

In [1]:
import os
from dotenv import load_dotenv
import openai
load_dotenv()

openai.api_key=os.getenv("OPENAI_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI


model=ChatGroq(model="deepseek-r1-distill-llama-70b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F51AFF2200>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F51AFF35B0>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain_core.messages import HumanMessage,SystemMessage
messages=[
  SystemMessage(content="Translate the following from English to French"),
  HumanMessage(content="Hello, How are you?")
]

response=model.invoke(messages)
response

AIMessage(content='<think>\nOkay, so I need to translate "Hello, How are you?" into French. Let me start by breaking down the sentence. "Hello" is a common greeting, and in French, the direct equivalent is "Bonjour." That\'s straightforward.\n\nNext, "How are you?" is a question asking about someone\'s well-being. In French, the common way to ask this is "Comment allez-vous?" which is formal, or "Comment ça va?" informally. Since the original sentence doesn\'t specify formality, I\'ll go with the formal version to keep it general.\n\nPutting it together, "Hello, How are you?" becomes "Bonjour, comment allez-vous?" I should also consider punctuation. The original has a comma after "Hello," so I\'ll include a comma after "Bonjour" as well.\n\nWait, maybe I should check if there\'s a more natural way to structure it in French. Sometimes greetings and questions are combined differently. But I think "Bonjour, comment allez-vous?" is the standard and most polite way to say it. I don\'t see a

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(response)

'<think>\nOkay, so I need to translate "Hello, How are you?" into French. Let me start by breaking down the sentence. "Hello" is a common greeting, and in French, the direct equivalent is "Bonjour." That\'s straightforward.\n\nNext, "How are you?" is a question asking about someone\'s well-being. In French, the common way to ask this is "Comment allez-vous?" which is formal, or "Comment ça va?" informally. Since the original sentence doesn\'t specify formality, I\'ll go with the formal version to keep it general.\n\nPutting it together, "Hello, How are you?" becomes "Bonjour, comment allez-vous?" I should also consider punctuation. The original has a comma after "Hello," so I\'ll include a comma after "Bonjour" as well.\n\nWait, maybe I should check if there\'s a more natural way to structure it in French. Sometimes greetings and questions are combined differently. But I think "Bonjour, comment allez-vous?" is the standard and most polite way to say it. I don\'t see any mistakes here, 

---
Using LCEL we can chain the components

In [13]:
chain=model|parser
chain.invoke(messages)

'<think>\nAlright, the user wants to translate "Hello, How are you?" into French. I know the basic greetings in French. "Hello" is "Bonjour." Now, for "How are you?" the common translation is "Comment ça va?" That should cover it. Let me put it together: "Bonjour, comment ça va?" That sounds natural and polite. I think that\'s the best way to translate it.\n</think>\n\nBonjour, comment ça va ?'

---
Using PromptTemplates

In [20]:
from langchain_core.prompts import ChatPromptTemplate
generic_template="Translate the following into {language}"

prompt=ChatPromptTemplate.from_messages(
  [
    ("system",generic_template),
    ("user","{text}")
  ]
)

In [21]:
res=prompt.invoke({"language":"French","text":"Hello"})

In [22]:
res.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

**Chaining components with LCEL** \
prompt->model->parser

In [23]:
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'<think>\n\n</think>\n\nBonjour'